# EVOLUCIÓN DIFERENCIAL

Determinar mediante la ED el mínimo global de la fucnción Easom en dos variables

$$f(x,y) = -\cos(x)\cos(y)e^{-(x-\pi)^2-(y-\pi)^2}$$

El espacio de busqueda será $(x,y) \in [-5,5]$ y el valor mínimo global es $f(\pi,\pi) = -1$.


In [9]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import  random

pio.renderers.default = "browser"  # también puedes probar "vscode"


def easom_function(x, y):
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi) ** 2 + (y - np.pi) ** 2))



max_ = 5
min_ = -max_

x = np.linspace(min_, max_, 100)
y = np.linspace(min_, max_, 100)
X,Y = np.meshgrid(x,y)
Z = easom_function(X,Y)




fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.show()

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

#FUNCIONES TÉRMICAS ε–NTU

def area_intercambio_termico(Di, L):
    return np.pi * Di * L

def coef_transf_calor(U=500.0):
    return U

def capacidad_calorifica(m, cp=4180.0):
    return m * cp

def capacidades_min_max(Ch, Cc):
    return min(Ch, Cc), max(Ch, Cc)

def razon_capacidades(Cmin, Cmax):
    return Cmin / Cmax

def numero_unidades_transf(U, A, Cmin):
    return (U * A) / Cmin

def eficiencia_intercambiador(NTU, Cr, tol=1e-9):
    x = NTU * (1.0 - Cr)
    if abs(1 - Cr) < tol:
        eta = NTU / (1.0 + NTU)
    else:
        eta = (1 - np.exp(-x)) / (1 - Cr * np.exp(-x))
    eta = max(min(eta, 1 - tol), tol)
    return eta

def calor_transferido(eta, Cmin, Th_in=80.0, Tc_in=20.0):
    return eta * Cmin * (Th_in - Tc_in)

def temperaturas_salida(Q, Ch, Cc, Th_in=80.0, Tc_in=20.0):
    Th_out = Th_in - Q / Ch
    Tc_out = Tc_in + Q / Cc
    return Th_out, Tc_out


def restricciones(Di, Do, L, m_hot, m_cold, Th_out, Tc_out, eta,
                  Th_in=80.0, Tc_in=20.0):
    ok = True
    msgs = []

    # === GEOMETRÍA ===
    if not (0.01 <= Di <= 0.05):
        ok = False; msgs.append("[Di] Diametro interior fuera de rango [0.01, 0.05]")
    if not (0.015 <= Do <= 0.06):
        ok = False; msgs.append("[Do] Diametro exterior fuera de rango [0.015, 0.06]")
    if not (Do > Di):
        ok = False; msgs.append("Geometría inválida: [Do] Diametro exterior debe ser > [Di] Diametro interior")
    if not (1.0 <= L <= 10.0):
        ok = False; msgs.append("[L] Longitud fuera de rango [1, 10]")

    # === FLUJOS ===
    if not (0.05 <= m_hot <= 0.5):
        ok = False; msgs.append("[ṁh] Flujo másico caliente fuera de rango [0.05, 0.5]")
    if not (0.05 <= m_cold <= 0.5):
        ok = False; msgs.append("[ṁc] Flujo másico frío fuera de rango [0.05, 0.5]")

    # === FÍSICA REAL ===
    if not (Th_out > Tc_in):
        ok = False; msgs.append("[Th_out] Temperatura salida caliente debe ser > [Tc_in] Temperatura entrada fría")

    if not (0 < eta < 1):
        ok = False; msgs.append("[eta] Eficiencia fuera de (0,1)")

    if not (Tc_out < Th_out):
        ok = False; msgs.append("[Tc_out] Temperatura salida fría no puede ser >= [Th_out] Temperatura salida caliente")

    if not (Tc_out <= Th_in):
        ok = False; msgs.append("[Tc_out] Temperatura salida fría no puede superar [Th_in] Temperatura entrada caliente")

    if not ((Th_out - Tc_out) > 0):
        ok = False; msgs.append("ΔT_out debe ser > 0")

    return ok, msgs



print(f"\n{"="*30} MEJOR DISEÑO ENCONTRADO {"="*30}\n")
print(f"[Di] Diametro interior = {Di:.6f} m")
print(f"[Do] Diametro exterior = {Do:.6f} m")
print(f"[L] Longitud          = {L:.6f} m")
print(f"[ṁh] Flujo másico caliente = {m_h:.6f} kg/s")
print(f"[ṁc] Flujo másico frío     = {m_c:.6f} kg/s")
print(f"\n{'='*30} Desempeño térmico {'='*30}\n")
print(f"[eta (ε)] Eficiencia del intercambiador = {eta:.6f}")
print(f"[NTU] Número de unidades de transferencia = {NTU:.6f}")
print(f"[Q] Calor transferido = {Q:.3f} W")
print(f"[Th_out] Temperatura salida caliente = {Th_out:.3f} °C")
print(f"[Tc_out] Temperatura salida fría = {Tc_out:.3f} °C")
print(f"[ΔT_out] Diferencia de temperatura salida = {Th_out - Tc_out:.3f} °C")
print(f"[Área A] Área de intercambio térmico = {A:.6f} m²")
print(f"[C_r] Razón de capacidades = {Cr:.6f}")
# FUNCIÓN OBJETIVO PARA ED

def fun_objetivo(X):
    Di, Do, L, m_h, m_c = X

    # Cálculos térmicos
    A = area_intercambio_termico(Di, L)
    U = coef_transf_calor(500)

    Ch = capacidad_calorifica(m_h)
    Cc = capacidad_calorifica(m_c)

    Cmin, Cmax = capacidades_min_max(Ch, Cc)
    if Cmin <= 0 or Cmax <= 0:
        return 1e9

    Cr  = razon_capacidades(Cmin, Cmax)
    NTU = numero_unidades_transf(U, A, Cmin)
    eta = eficiencia_intercambiador(NTU, Cr)

    Q = calor_transferido(eta, Cmin)
    Th_out, Tc_out = temperaturas_salida(Q, Ch, Cc)

    ok, msgs = restricciones(Di, Do, L, m_h, m_c, Th_out, Tc_out, eta)
    if not ok:
        return 1e6

    return -eta


In [10]:
# INICIALIZACIÓN DE POBLACION
def iniciar_poblacion(tamaño_poblacion : int,tamaño_individuo : int, Imin, Imax):
    poblacion = np.zeros((tamaño_poblacion, tamaño_individuo))
    for i in range(tamaño_poblacion):
        individuo = np.zeros((tamaño_individuo))
        for j in range(tamaño_individuo):
            individuo[j] = random.uniform(Imin, Imax)
        poblacion[i] = individuo
    return poblacion

In [11]:
# FUNCION DE EVALUACION
def evaluar(poblacion):
    n = len(poblacion)
    fitness = np.zeros((n))
    for i in range(n):
        x = poblacion[i][0]
        y = poblacion[i][1]
        fitness[i] = easom_function(x, y)
    return fitness

In [12]:
# FUNCION DE MUTACION
def mutacion(poblacion, tasa_mutacion,indice):
    F = tasa_mutacion
    n = len(poblacion)
    indices = list(range(n))
    indices.remove(indice)

    r1 = random.choice(indices)
    indices.remove(r1)
    r2 = random.choice(indices)
    indices.remove(r2)
    r3 = random.choice(indices)
    
    
    vector_mutante = np.zeros((poblacion.shape[1]))
    
    for i in range(poblacion.shape[1]):
        vector_mutante[i] = poblacion[r1][i] + F * (poblacion[r2][i] - poblacion[r3][i])
        
    return vector_mutante

In [13]:
# FUNCION DE CRUZA
def cruza(individuo, vector_mutante, tasa_cruza):
    CR = tasa_cruza
    n = len(individuo)
    vector_prueba = np.zeros((n))
    j_rand = random.randint(0, n-1)
    for i in range(n):
        if random.random()  < CR or i == j_rand:
            vector_prueba[i]  = vector_mutante[i]
        else:
            vector_prueba[i] = individuo[i]
    return vector_prueba

In [14]:
# FUNCION SELECCION
def seleccion(individuo, vector_prueba):
    f1 = easom_function(individuo[0], individuo[1])
    f2 = easom_function(vector_prueba[0], vector_prueba[1])
    if f2 < f1:
        return vector_prueba
    else:
        return individuo

In [15]:
# ================= IMPLEMENTACION DEL ALGORITMO =================
tamaño_poblacion = 100
Imax = 5
Imin = -Imax
F = 0.8
CR = 0.9
generaciones = 100


hisotrico_fitness = []


poblacion = iniciar_poblacion(tamaño_poblacion, 2, Imin, Imax)
for g in range(generaciones):
    nueva_poblacion = np.zeros((tamaño_poblacion, 2))
    for i in range(tamaño_poblacion):
        mutacion_vector = mutacion(poblacion, F,i)
        vector_prueba = cruza(poblacion[i], mutacion_vector, CR)
        nuevo_individuo = seleccion(poblacion[i], vector_prueba)
        nueva_poblacion[i] = nuevo_individuo
    poblacion = nueva_poblacion
    mejor_fitness = np.min(evaluar(poblacion))
    mejor_fitness_indice = np.argmin(evaluar(poblacion))
    mejor_valor = poblacion[mejor_fitness_indice]
    hisotrico_fitness.append(mejor_fitness)
    
    print(f"Generacion {g+1}, Mejor fitness: {mejor_fitness} en {mejor_valor}")

Generacion 1, Mejor fitness: -0.6662576567025734 en [3.27200613 2.64161927]
Generacion 2, Mejor fitness: -0.7416917830948299 en [2.94093366 2.74482849]
Generacion 3, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 4, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 5, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 6, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 7, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 8, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 9, Mejor fitness: -0.7966605819614186 en [3.47566361 3.33945198]
Generacion 10, Mejor fitness: -0.9289991677836607 en [3.35375624 3.20460132]
Generacion 11, Mejor fitness: -0.9289991677836607 en [3.35375624 3.20460132]
Generacion 12, Mejor fitness: -0.9289991677836607 en [3.35375624 3.20460132]
Generacion 13, Mejor fitness: -0.9818756005322848 en [3.24480208 3.10241187]
Generaci

In [16]:
# Grafica de la evolucion del fitness
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=hisotrico_fitness, mode='lines+markers', name='Mejor Fitness'))
fig2.update_layout(title='Evolución del Mejor Fitness',
                   xaxis_title='Generación',
                   yaxis_title='Mejor Fitness')
fig2.show()